In [30]:
from numpy import *
from scipy.special import jn, yn, jvp, yvp, jn_zeros
from scipy.constants import c, epsilon_0, mu_0
from scipy.optimize import brentq


In [6]:
def f0(R0):
    # unperturbed frequency
    # Pozar pp 290, 
    # TM(n, m, l): f = c/(2*pi) * sqrt( (p_nm/a)**2 + (l*pi/d)**2)
    # p_nm - m-th root of J_n
    p_01 = jn_zeros(0, 1)[0]
    f = c/(2*pi) * p_01/R0
    return f

In [12]:
def f_perturbed(R0, R1, eps_r):
    # simple perturbation theory for solid rod inserted, Li, Akyel, Bosisio;
    # Precise Calculations and Measurements on the Complex Dielectric Constant of Lossy
    # Materials Using TMOIO Cavity Perturbation Techniques, IEEE, 1981:
    # eps_p - 1 = 2*J_1^2(x_01)*df/f * R0^2/R1^2
    p_01 = jn_zeros(0, 1)[0]
    df_rel = (eps_r-1) / (2*jn(1, p_01)**2) * R1**2/R0**2
    f = f0(R0)*(1-df_rel)
    return f

In [45]:
def f_Li(R0, R1, R2, eps_r, eps_g):
    # Li, Lukac theory:
    eps_1 = eps_r
    eps_2 = eps_g
    def eq(fs):
        k0 = 2*pi*fs/c
        k = [k0, sqrt(eps_1)*k0, sqrt(eps_2)*k0]
        R = [R0, R1, R2]
        def J(n, p, q): return jn(n, k[p]*R[q])
        def Y(n, p, q): return yn(n, k[p]*R[q])
        A = sqrt(eps_2) * (J(0,0,0)*Y(0,0,2) - J(0,0,2)*Y(0,0,0)) / (J(0,0,0)*Y(1,0,2) - J(1,0,2)*Y(0,0,0))
        nom = J(1,2,1)*Y(0,2,2) - Y(1,2,1)*J(0,2,2) - A*(J(1,2,1)*Y(1,2,2) - Y(1,2,1)*J(1,2,2))
        den = J(0,2,1)*Y(0,2,2) - Y(0,2,1)*J(0,2,2) - A*(J(0,2,1)*Y(1,2,2) - Y(0,2,1)*J(1,2,2))
        F = sqrt(eps_2)*k0*nom/den
        lhs = k[1]*J(1,1,1)/J(0,1,1)
        return lhs-F
    
    from scipy.optimize import brentq
    f = brentq(eq, f0(R0)*0.5, f0(R0)*1.2)
    return f


In [49]:
# dielectric sample inserted in TM010 cavity
# n, m, l - angular, radial, axial
n, m, l = 0, 1, 0

f_target = 2.45e9
f_measured = 2.55e9

### Empty cavity

In [35]:
# calculate optimal cavity radius
R0 = brentq(lambda x: f0(x)-f_target, 30e-3, 70e-3)
R1 = 11.0e-3
R2 = 12.5e-3 # containing tube radius
eps_r = 1
eps_g = 10


print("Radius of empty cavity for 2.45 GHz")
print("r = %f mm" % (R0*1e3))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

Radius of empty cavity for 2.45 GHz
r = 46.833685 mm
Unperturbed frequency
f = 2.450000 GHz
Frequency with tube
f = 2.007178 GHz


### Test cavity design

In [37]:
R0 = 36.0e-3
R1 = 11.0e-3
R2 = 12.5e-3 # containing tube radius
eps_r = 1
eps_g = 10

print("Radius of test cavity")
print("r = %f mm" % (R0*1e3))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

Radius of test cavity
r = 36.000000 mm
Unperturbed frequency
f = 3.187292 GHz
Frequency with tube
f = 2.436911 GHz


### Test cavity - actual sapphire permittivity?

In [53]:
R0 = 36.0e-3
R1 = 11.0e-3
R2 = 12.5e-3 # containing tube radius
eps_r = 1
eps_g = brentq(lambda x: f_Li(R0, R1, R2, eps_r, x)-f_measured, 5, 15)
#print(f_Li(R0, R1, R2, eps_r, 15)/1e9)
print("Sapphire permittivity to match the measured frequency")
print("eps = %f" % (eps_g))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

print()
# corrected cavity radius to match the target
R0 = brentq(lambda x: f_Li(x, R1, R2, eps_r, eps_g)-f_target, 30e-3, 50e-3)
print("Corrected cavity radius to match the target frequency")
print("R0 = %f mm" % (R0*1e3))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

Sapphire permittivity to match the measured frequency
eps = 8.189117
Unperturbed frequency
f = 3.187292 GHz
Frequency with tube
f = 2.550000 GHz

Corrected cavity radius to match the target frequency
R0 = 37.918811 mm
Unperturbed frequency
f = 3.026005 GHz
Frequency with tube
f = 2.450000 GHz


### Test cavity - actual sapphire outer radius?

In [65]:
eps_r = 1
eps_g = 10
R0 = 36.0e-3
R1 = 11.0e-3
R2 = brentq(lambda x: f_Li(R0, R1, x, eps_r, eps_g)-f_measured, R1, R1*1.5)

print("Sapphire outer radius to match the measured frequency")
print("R2 = %f mm" % (R2*1e3))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

print()
# corrected cavity radius to match the target
R0 = brentq(lambda x: f_Li(x, R1, R2, eps_r, eps_g)-f_target, 30e-3, 50e-3)
print("Corrected cavity radius to match the target frequency")
print("R0 = %f mm" % (R0*1e3))

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

Sapphire outer radius to match the measured frequency
R2 = 12.199596 mm
Unperturbed frequency
f = 3.187292 GHz
Frequency with tube
f = 2.550000 GHz

Corrected cavity radius to match the target frequency
R0 = 37.926949 mm
Unperturbed frequency
f = 3.025356 GHz
Frequency with tube
f = 2.450000 GHz


In [67]:

# http://www.crytur.cz/materials/sapphire/
# eps = 11.5 parallel to c axis
# eps = 9.3 perpendicular to c axis
# assume cylindrical symmetry
# tube outer radius: 25.3 -- 25.05 mm
# tube inner radius 22.20 mm
eps_r = 1

eps_g = 11.5
R0 = 36.0e-3
R1 = 11.1e-3
R2 = 12.69e-3

print("Unperturbed frequency")
print("f = %f GHz" % (f0(R0)/1e9))

print("Frequency with tube")
print("f = %f GHz" % (f_Li(R0, R1, R2, eps_r, eps_g)/1e9))

Unperturbed frequency
f = 3.187292 GHz
Frequency with tube
f = 2.319312 GHz
